In [1]:
import numpy as np
import pandas as pd
import time
import re
import matplotlib.pyplot as plt
import scipy.sparse as sps
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import StandardScaler
import operator
from collections import OrderedDict
import nltk
from nltk.corpus import stopwords
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.probability import FreqDist, ConditionalFreqDist
from importlib import reload

In [2]:
%matplotlib inline

In [3]:
train_path = 'train-contest.csv'
data = pd.read_csv(train_path)

In [733]:
# data.to_csv("new_data.csv", columns=['PartsOfSpeach'])

In [4]:
new_data = pd.read_csv("new_data.csv", parse_dates=True, dayfirst=True)

In [5]:
reasons = pd.read_csv('reason.csv')

In [6]:
data['Reason'] = reasons['OpenStatus']

In [7]:
data['PartsOfSpeach'] = new_data['PartsOfSpeach']

In [8]:
data.PartsOfSpeach[0]

"['IN', 'NNP', 'NNP', 'NN', 'NN', 'PRP', 'VBP', 'VBN', 'JJ', 'NN', 'NNP', 'DT', 'NNP', 'NN', 'IN', 'JJ', 'NN', 'NN', 'IN', 'PRP$', 'NNP', 'NN', 'NN', 'PRP', 'VBP', 'VBN', 'NNP', 'NNP', 'VBP', 'RB', 'VB', 'DT', 'NNP', 'NN', 'CC', 'IN', 'WP', 'PRP', 'VBN', 'NN', 'NN', 'VBP', 'VBZ', 'RP', 'NN', 'IN', 'NN', 'PRP', 'VBP', 'NN', 'IN', 'NN', 'CC', 'PRP', 'MD', 'RB', 'VB', 'PRP', 'IN', 'DT', 'JJ', 'NN', 'NN', 'NN', 'CC', 'DT', 'NN', 'NN', 'IN', 'NNP', 'PRP', 'MD', 'VB', 'CD', 'NN', 'IN', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NNP', 'NNP', 'NN', 'CC', 'PRP', 'MD', 'VB', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN', 'CC', 'DT', 'JJ', 'NN', 'PRP', 'CC', 'VB', 'DT', 'JJ', 'NN', 'IN', 'DT', 'NN', 'NNP', 'DT', 'NN', 'MD', 'VB', 'VB', 'PRP', 'TO', 'DT', 'NN', 'NN', 'CC', 'IN', 'PRP', 'VBD', 'RB', 'VB', 'PRP', 'MD', 'VB', 'DT', 'NN', 'IN', 'PRP', 'PRP', 'VBP', 'IN', 'DT', 'NNP', 'NN', 'IN', 'NNP', 'CC', 'PRP', 'VBP', 'IN', 'NNP', 'RB', 'VBD', 'DT', 'NNP', 'NN', 'WDT', 'VBP', 'DT', 'NN', 'IN', 'NN', 'IN', 'DT', 'J

#Извлечение информации

Разбиваем Body на текст и код

In [9]:
data['Code'] = data.BodyMarkdown.apply(lambda text: " ".join([line for line in text.split('\n') if 
                                                     line.startswith('    ') or line.startswith('\t')]))
data['Text'] = data.BodyMarkdown.apply(lambda text: " ".join([line for line in text.split('\n') if not
                                                     line.startswith('    ') and not line.startswith('\t')]))

In [10]:
data['SplittedCode'] = data.Code.apply(lambda code: re.sub("[^a-z]", " ", code.lower()).split())

In [11]:
data['CodeLen'] = data.SplittedCode.apply(len)

##Текст

Несколько статистик о тексте

In [12]:
# data.Text = data.Text.apply(lambda text: ' '.join(text))
data['NonWords'] = data.Text.apply(lambda text: len(re.findall('[A-Za-z\d_]+[ \.\?\,\!]', text)))
data['Digits'] = data.Text.apply(lambda text: len(re.findall('\d+', text)))
data['NonAlNums'] = data.Text.apply(lambda text: len(re.findall('[^\w\s]+', text)))
data['URLs'] = data.Text.apply(lambda text: len(re.findall('https?://', text)))

In [13]:
data['Text'] = data.Text.apply(nltk.word_tokenize)

In [14]:
data['PureText'] = data.Text.apply(lambda text: [word for word in text if word[0].isalpha()])

In [15]:
data['NumWords'] = data.PureText.apply(lambda x: len(x))

In [16]:
data.Title = data.Title.apply(nltk.word_tokenize)

In [17]:
data['Sentences'] = data.Text.apply(lambda x:nltk.sent_tokenize(" ".join(x)))

In [18]:
data['NumSentences'] = data.Sentences.apply(len)

In [19]:
data['Questions'] = data.Sentences.apply(lambda x: len([sent for sent in x if sent[-1]=='?']))

In [20]:
data['Questions_ratio'] = data.Questions / data.NumSentences
data.Questions_ratio = data.Questions_ratio.fillna(0)

In [21]:
data['NumLastQuestions'] = data.Sentences.apply(lambda text:
                    [sent[-1] for sent in text[-5:]].count('?'))

In [22]:
data.Questions_ratio[0]

0.33333333333333331

In [23]:
" ".join(data.Text[0])

"In My VB.NET web page , I have this standard event . Note the `` Handles '' clause on teh event declaration . In my C # web app , I have this : Since C # does n't have a `` Handles '' equivalent and from what I 've seen , event handlers are wired up using delegate += syntax , I was looking for this , but I could not foind it in the aspx page , aspx.cs file or the aspx.designer.cs file . In VB , I would have two drop down lists at the top of the Code Editor window and I could select any object on the web form or the web form itself and see the possible events for the object . Selecting the event would either take me to the event handler or if it did n't exists , it would create the stub for me . I know that the Properties window in C # ( and I think in VB , too ) has an Event tab that shows the list of events for the selected object GUI object , but `` Page '' does n't appear as an object that can be selected . 1 . Where does C # define the hooking up of the event to the handler ? 2 . 

##Пользователь

In [24]:
data['PostCreationDate'] = data['PostCreationDate'].apply(lambda x: time.strptime(x[:10], "%m/%d/%Y"))
data['OwnerCreationDate'] = data['OwnerCreationDate'].apply(lambda x: x if (x != '2008-09-01') else '01/09/2008')
data['OwnerCreationDate'] = data['OwnerCreationDate'].apply(lambda x: time.strptime(x[:10], "%m/%d/%Y"))
data['OwnerCreationDate'] = data['OwnerCreationDate'].apply(lambda x: time.mktime(x) / 86400)
data['PostCreationDate'] = data['PostCreationDate'].apply(lambda x: time.mktime(x) / 86400)
data['DaysBeforePost'] = data['PostCreationDate'] - data['OwnerCreationDate']
data[data['DaysBeforePost'] < 0].DaysBeforePost = -1

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/pandas/core/generic.py:2177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


##Теги

In [25]:
data['Tags'] = list(zip(data.Tag1.fillna(""), 
                               data.Tag2.fillna(""), data.Tag3.fillna(""), 
                               data.Tag4.fillna(""), data.Tag5.fillna(""))) 
data['Tags'] = data['Tags'].apply(lambda x: (" ".join(x).lower().split()))
data['EmptyTags'] = data['Tags'].apply(lambda tagset: len(tagset))

#Разобьем данные на тестовую и обучающую выборки

In [141]:
train_data = data[:70000]
test_data = data[70000:]
train_columns = [col for col in data.columns if col != 'OpenStatus' and col != 'PostClosedDate']
x_tr = train_data[train_columns]
y_tr = train_data.OpenStatus
x_test = test_data[train_columns]
y_test = test_data.OpenStatus

#TF-IDF текстов

In [27]:
vect = TfidfVectorizer(lowercase=False, tokenizer=lambda doc: doc)
text_tr_tfidf = vect.fit_transform(x_tr.Text)
text_test_tfidf = vect.transform(x_test.Text)
title_tr_tfidf = vect.fit_transform(x_tr.Title)
title_test_tfidf = vect.transform(x_test.Title)
tag_tr_tfidf = vect.fit_transform(x_tr.Tags)
tag_test_tfidf = vect.transform(x_test.Tags)
code_tr_tfidf = vect.fit_transform(x_tr.SplittedCode)
code_test_tfidf = vect.transform(x_test.SplittedCode)

In [142]:
x_tr.Title = x_tr.Title.apply(" ".join)
x_test.Title = x_test.Title.apply(" ".join)
bigram_vectorizer = CountVectorizer(ngram_range=(2,2), token_pattern=r'\b\w+\b', min_df=1)
bititle_tr = bigram_vectorizer.fit_transform(x_tr.Title)
bititle_test = bigram_vectorizer.transform(x_test.Title)
transformer = TfidfTransformer()
bititle_test = transformer.fit_transform(bititle_test)
bititle_tr = transformer.fit_transform(bititle_tr)
x_tr.Title = x_tr.Title.apply(lambda text: text.split())
x_test.Title = x_test.Title.apply(lambda text: text.split())

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/pandas/core/generic.py:2177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [28]:
x_tr.PureText = x_tr.PureText.apply(" ".join)
x_test.PureText = x_test.PureText.apply(" ".join)
bigram_vectorizer = CountVectorizer(ngram_range=(2,2), token_pattern=r'\b\w+\b', min_df=1)
bigrams_tr = bigram_vectorizer.fit_transform(x_tr.PureText)
bigrams_test = bigram_vectorizer.transform(x_test.PureText)
transformer = TfidfTransformer()
bigrams_test = transformer.fit_transform(bigrams_test)
bigrams_tr = transformer.fit_transform(bigrams_tr)
x_tr.PureText = x_tr.PureText.apply(lambda text: text.split())
x_test.PureText = x_test.PureText.apply(lambda text: text.split())

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/pandas/core/generic.py:2177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [126]:
vect = TfidfVectorizer(lowercase=False, tokenizer=lambda doc: doc)
pos_tr_tfidf = vect.fit_transform(x_tr.PartsOfSpeach)
pos_test_tfidf = vect.transform(x_test.PartsOfSpeach)

In [149]:
x_tr.PartsOfSpeach = x_tr.PartsOfSpeach.apply(" ".join)
x_test.PartsOfSpeach = x_test.PartsOfSpeach.apply(" ".join)
bigram_vectorizer = CountVectorizer(ngram_range=(2,2), token_pattern=r'.', min_df=1)
bipos_tr = bigram_vectorizer.fit_transform(x_tr.PartsOfSpeach)
bipos_test = bigram_vectorizer.transform(x_test.PartsOfSpeach)
transformer = TfidfTransformer()
bipos_test = transformer.fit_transform(bipos_test)
bipos_tr = transformer.fit_transform(bipos_tr)
x_tr.PartsOfSpeach = x_tr.PartsOfSpeach.apply(lambda text: text.split())
x_test.PartsOfSpeach = x_test.PartsOfSpeach.apply(lambda text: text.split())

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/pandas/core/generic.py:2177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [150]:
print(tag_tr_tfidf.shape)
print(title_tr_tfidf.shape)
print(text_tr_tfidf.shape)
print(code_tr_tfidf.shape)
print(bigrams_tr.shape)
print(pos_tr_tfidf.shape)
print(bipos_tr.shape)
print(bititle_tr.shape)

(70000, 13938)
(70000, 43930)
(70000, 220561)
(70000, 112251)
(70000, 1090637)
(70000, 30)
(70000, 57)
(70000, 221033)


#Функции для шкалирования

In [65]:
def log_scaler(df):
    new_df = df.copy()
    quant = new_df.quantile(0.9)
    if new_df.min() < 0:
        new_df -= new_df.min()
    new_df[new_df > quant] = np.log(new_df[new_df > quant] - quant + 1) + quant
    return 0.5 + (new_df - new_df.mean()) / (new_df.std())

def norm_scaler(df):
    new_df = df.copy()
    return 0.5 + (new_df - new_df.mean()) / new_df.std()

def is_very_big(df):
    new_df = df.copy()
    quant = new_df.quantile(0.95)
    new_df[new_df > quant] = 1
    new_df[new_df < quant] = 0
    return new_df

In [33]:
norm_scaler(data.Questions_ratio).describe()

count    100000.000000
mean          0.500000
std           1.000000
min          -0.445222
25%          -0.445222
50%           0.286462
75%           1.018146
max           3.213197
Name: Questions_ratio, dtype: float64

 #Поэкспериментируем с текстом

In [35]:
data['Excls_ratio'] = data.Sentences.apply(lambda text:
                    [sent[-1] for sent in text].count('!')) / (data.NumWords+1)
data['Statements_ratio'] = data.Sentences.apply(lambda text:
                    [sent[-1] for sent in text].count('.')) / (data.NumWords+1)
data.Statements_ratio = data.Statements_ratio.fillna(0)
data.Excls_ratio = data.Excls_ratio.fillna(0)

In [36]:
data['Comma_ratio'] = data.Text.apply(lambda text: text.count(',')) / (data.NumWords+1)

In [37]:
data['AverageWordLen'] = data.Text.apply(lambda x: len(" ".join(x))) / (data.NumWords+1)

In [38]:
data['AverageSentLen'] = data.Sentences.apply(len) / (data.NumWords+1)

In [39]:
# data['PartsOfSpeach'] = data.PureText.apply(lambda text: [word[1] for word in nltk.pos_tag(text)])

In [40]:
data['PartsOfSpeach'].head()

0    ['IN', 'NNP', 'NNP', 'NN', 'NN', 'PRP', 'VBP',...
1    ['PRP', 'VBP', 'TO', 'VB', 'NNP', 'IN', 'NNP',...
2    ['PRP$', 'NNP', 'NNP', 'VBZ', 'NN', 'IN', 'NNP...
3    ['PRP', 'VBP', 'DT', 'JJ', 'NNP', 'NN', 'NN', ...
4    ['NN', 'NN', 'TO', 'VB', 'JJ', 'NN', 'IN', 'NN...
Name: PartsOfSpeach, dtype: object

In [43]:
train_data = data[:70000]
test_data = data[70000:]
train_columns = [col for col in data.columns if col != 'OpenStatus' and col != 'PostClosedDate']
x_tr = train_data[train_columns]
y_tr = train_data.OpenStatus
x_test = test_data[train_columns]
y_test = test_data.OpenStatus

#Попробуем вычислить попарные произведения признаков

In [170]:
def calculate_products(x):
    dim = x.shape[1]
    print(dim)
    for i in range(dim):
        for j in range(dim):
#             print(i, j)
            x = np.hstack((x, (x[:, i] * x[:, j])[:, None]))
    return x

#Собираем признаки

In [185]:
cols = ['CodeLen', 'NumWords', 'DaysBeforePost', 'NumSentences', 'Questions_ratio',  'NonWords', 'Digits', 
             'NonAlNums', 'EmptyTags', 'NumLastQuestions', 'ReputationAtPostCreation', 
             'OwnerUndeletedAnswerCountAtPostTime', 'Comma_ratio']
cols_norm = cols
# cols_norm = ['CodeLen']
# cols_log = ['DaysBeforePost', 'ReputationAtPostCreation', 'OwnerUndeletedAnswerCountAtPostTime', ]
cols_log = cols
# cols_log =[]

x_train_2 = np.array(norm_scaler(x_tr[cols_norm[0]]))[:, None]
x_test_2 = np.array(norm_scaler(x_test[cols_norm[0]]))[:, None]
for col in cols_norm[1:]:
    x_train_2 = np.hstack((x_train_2, np.array(norm_scaler(x_tr[col]))[:, None]))
    x_test_2 = np.hstack((x_test_2, np.array(norm_scaler(x_test[col]))[:, None]))
for col in cols_log:
    x_train_2 = np.hstack((x_train_2, np.array(log_scaler(x_tr[col]))[:, None]))
    x_test_2 = np.hstack((x_test_2, np.array(log_scaler(x_test[col]))[:, None]))
# for col in cols_big:
#     x_train_2 = np.hstack((x_train_2, np.array(is_very_big(x_tr[col]))[:, None]))
#     x_test_2 = np.hstack((x_test_2, np.array(is_very_big(x_test[col]))[:, None]))
print(np.sum(np.isnan(x_test_2)))

0


In [186]:
# x_train_2 = sps.csr_matrix(sps.hstack((x_train_2, text_tr_tfidf, title_tr_tfidf, tag_tr_tfidf)))
# x_test_2 = sps.csr_matrix(sps.hstack((x_test_2, text_test_tfidf, title_test_tfidf, tag_test_tfidf)))
x_train_2 = sps.csr_matrix(sps.hstack((x_train_2, tag_tr_tfidf, title_tr_tfidf, text_tr_tfidf, code_tr_tfidf, 
                                       small_bigrams_tr_tfidf, bititle_tr)))
x_test_2 = sps.csr_matrix(sps.hstack((x_test_2, tag_test_tfidf, title_test_tfidf, text_test_tfidf, code_test_tfidf, 
                                      small_bigrams_test_tfidf, bititle_test)))

In [187]:
clf_linear = LogisticRegression()
clf_linear.fit(x_train_2, y_tr)
predicted_y_test = clf_linear.predict_proba(x_test_2)
print(roc_auc_score(y_test, predicted_y_test[:, 1]))

0.850614880441


In [179]:
x_train_2.shape

(70000, 68)

In [409]:
small_bigrams_test_tfidf.shape

(30000, 500000)

In [259]:
x_test[y_test != np.round(predicted_y_test[:, 1])].Reason.value_counts()

open                   2990
not a real question    1970
off topic               943
not constructive        760
too localized           443
dtype: int64

#Random Forest

In [224]:
cols = ['CodeLen', 'NumWords', 'DaysBeforePost', 'NumSentences', 'Questions_ratio',  'NonWords', 'Digits', 
             'NonAlNums', 'EmptyTags', 'NumLastQuestions', 'ReputationAtPostCreation', 
             'OwnerUndeletedAnswerCountAtPostTime', 'Comma_ratio']
cols_norm = cols
cols_log = cols

x_train_knn = np.array(norm_scaler(x_tr[cols_norm[0]]))[:, None]
x_test_knn = np.array(norm_scaler(x_test[cols_norm[0]]))[:, None]
for col in cols_norm[1:]:
    x_train_knn = np.hstack((x_train_knn, np.array(norm_scaler(x_tr[col]))[:, None]))
    x_test_knn = np.hstack((x_test_knn, np.array(norm_scaler(x_test[col]))[:, None]))
for col in cols_log:
    x_train_knn = np.hstack((x_train_knn, np.array(log_scaler(x_tr[col]))[:, None]))
    x_test_knn = np.hstack((x_test_knn, np.array(log_scaler(x_test[col]))[:, None]))

In [225]:
x_train_knn = sps.csr_matrix(sps.hstack((x_train_knn, small_tag_tr_tfidf, small_text_tr_tfidf, 
                                         small_title_tr_tfidf, small_code_tr_tfidf, small_bigrams_tr_tfidf)))
x_test_knn = sps.csr_matrix(sps.hstack((x_test_knn, small_tag_test_tfidf, small_text_test_tfidf,
                                       small_title_test_tfidf, small_code_test_tfidf, small_bigrams_test_tfidf)))

In [226]:
x_test_knn.shape

(30000, 8026)

In [281]:
clf = RandomForestClassifier(n_estimators=700, n_jobs=-1)
clf.fit(x_train_knn, y_tr)
forest_prediction = clf.predict_proba(x_test_knn)
forest_prediction_saver = np.copy(forest_prediction)
print(roc_auc_score(y_test, forest_prediction[:, 1]))

KeyboardInterrupt: 

In [701]:
n_list = [100, 500, 700, 800, 1000, 1100]
start = time.time()
for n in n_list:
    clf = RandomForestClassifier(n_estimators=n, n_jobs=-1)
    clf.fit(x_train_knn, y_tr)
    forest_prediction = clf.predict_proba(x_test_knn)
    print(roc_auc_score(y_test, forest_prediction[:, 1]))
    print(time.time() - start)

0.809455254504
163.25106382369995
0.818723316739
1063.5971138477325
0.818734683422
2344.9134528636932
0.819456873351
3795.4877338409424
0.819156810697
5641.991401910782
0.82004473642
7642.447065830231


In [228]:
for w in np.linspace(0, 1, 10):
    print(roc_auc_score(y_test, w * forest_prediction[:, 1] + (1 - w) * predicted_y_test[:, 1]))

0.850614880441
0.852528938753
0.853896487389
0.854528161632
0.854116227705
0.852262633925
0.848393799465
0.841868418957
0.832029009233
0.818387747367


#Исследуем ошибки

In [188]:
def plot_results(prediction):
    reasons_lst = ['open', 'not a real question', 'off topic', 'not constructive', 'too localized']
#     print(len(x_test[y_test != np.round(prediction[:, 1])][x_test.Reason == 'open']))
    sizes = [len(x_test[(y_test != np.round(prediction[:, 1])) & 
                        (x_test.Reason == reason)])/len(y_test[x_test.Reason == reason])
             for reason in reasons_lst]
    colors = ['yellowgreen', 'gold', 'lightskyblue', 'lightcoral', 'red']
    explode = (0, 0, 0, 0, 0)  # only "explode" the 2nd slice (i.e. 'Hogs')
#     labels = reasons_lst

#     plt.pie(sizes, explode=explode, labels=labels, colors=colors,
#         autopct='%1.1f%%', shadow=True, startangle=90)
    for i in range(5):
        print(reasons_lst[i],':', sizes[i])
#     plt.bar(range(5), sizes)
#     plt.xticks([0.5, 1.5, 2.5, 3.5, 4.5], reasons_lst, rotation='vertical')
#     plt.show()

In [229]:
plot_results(forest_prediction)

open : 0.25455405725100544
not a real question : 0.2559865900383142
off topic : 0.25399201596806387
not constructive : 0.24653113907712165
too localized : 0.25161290322580643


In [646]:
plot_results(0.7 * predicted_y_test + 0.3*forest_prediction)

open : 0.22513997318823437
not a real question : 0.22629310344827586
off topic : 0.21756487025948104
not constructive : 0.22587931590835753
too localized : 0.22688172043010751


In [189]:
plot_results(predicted_y_test)

open : 0.22884630549641197
not a real question : 0.22796934865900384
off topic : 0.22080838323353294
not constructive : 0.2281381090674411
too localized : 0.22365591397849463


In [653]:
plot_results(text_predicted_y_test)

open : 0.24414478353442157
not a real question : 0.24113984674329503
off topic : 0.2465069860279441
not constructive : 0.24298160696999033
too localized : 0.24193548387096775


In [265]:
np.mean(predicted_y_test[(0 == np.round(predicted_y_test[:, 1]))][:, 1])

0.22212607892025807

In [266]:
noquest = x_test[(y_test != np.round(predicted_y_test[:, 1])) & (x_test.Reason == 'not constructive')] 
# noquest = x_test[x_test.Reason == 'not a real question']
noquest.index = range(len(noquest))
for i in range(10, 20): 
    print((predicted_y_test[(y_test != np.round(predicted_y_test[:, 1])) & (x_test.Reason == 'not constructive')])[i])
    print(i)
    print(noquest.ReputationAtPostCreation[i], noquest.DaysBeforePost[i])
    print(' '.join(noquest.Text[i]))
    print(noquest.Code[i])
# print(noquest.Questions_ratio.describe())

[ 0.60547104  0.39452896]
10
1384 324.0
I 'm tearing my hair out . Apparently you ca n't just do something like I 've searched the internet and I ca n't find a simple example of how to implement the damn thing ANYWHERE . Even MSDN . It 's ridiculous .
     }
[ 0.60547104  0.39452896]
11
3 2.0
I searched a lot but could n't find the way to measure web page loading time with iOS . In the app , I want to show certain page loading time.. Is it possible with iOS sdk or third party sdk ? Thanks

[ 0.60547104  0.39452896]
12
97 318.0
I have downloaded 'Adobe Flash builder for Force.com ' from Salesforce site and trying to get some hands on . If any one has tried it before request you to please share your experience . Is there seom kind of good tutorial available that shows step by step procedure to create flex SFDC projects in eclipse ( Flas Builder 4 plugin versoin ) from scratch .

[ 0.60547104  0.39452896]
13
1 48.9583333333
Hey , So i have about 4-5 years of background in programming some

/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/IPython/kernel/__main__.py:5: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index


In [200]:
noquest.ReputationAtPostCreation.describe()

count      707.000000
mean       301.950495
std        950.127468
min          1.000000
25%          1.000000
50%         25.000000
75%        179.500000
max      13310.000000
Name: ReputationAtPostCreation, dtype: float64

In [201]:
data.ReputationAtPostCreation.describe()

count    100000.00000
mean        512.30627
std        2572.75879
min         -34.00000
25%           1.00000
50%          36.00000
75%         267.00000
max      209631.00000
Name: ReputationAtPostCreation, dtype: float64

In [239]:
len(y_test[(y_test != np.round(predicted_y_test[:, 1])) & 
           (x_test.Reason == 'not constructive')]) / len(x_test[x_test.Reason == 'not constructive'])

0.2281381090674411

In [238]:
len(y_test[(y_test != np.round(predicted_y_test[:, 1])) & (y_test != np.round(forest_prediction[:, 1]))&
                 (x_test.Reason == 'not constructive')]) / len(x_test[x_test.Reason == 'not constructive'])

0.15392061955469508

In [269]:
len(y_test[(y_test != np.round(new_predictor)) & 
           (x_test.Reason == 'not constructive')]) / len(x_test[x_test.Reason == 'not constructive'])

0.2223297838012262

In [279]:
for threshold in np.linspace(0, 0.5, 20):
    new_predictor = forest_prediction[:, 1]
    new_predictor[np.abs(predicted_y_test[:, 1] - 0.5) > threshold] = \
        (predicted_y_test[:, 1])[np.abs(predicted_y_test[:, 1] - 0.5) > threshold]
    print(threshold, ":", roc_auc_score(y_test, new_predictor))
# new_predictor = np.max((np.abs(predicted_y_test[:, 1] - 0.5), np.abs(forest_prediction[:, 1]- 0.5)),axis=1)

0.0 : 0.850614880441
0.0263157894737 : 0.850614880441
0.0526315789474 : 0.850614880441
0.0789473684211 : 0.850614880441
0.105263157895 : 0.850614880441
0.131578947368 : 0.850614880441
0.157894736842 : 0.850614880441
0.184210526316 : 0.850614880441
0.210526315789 : 0.850614880441
0.236842105263 : 0.850614880441
0.263157894737 : 0.850614880441
0.289473684211 : 0.850614880441
0.315789473684 : 0.850614880441
0.342105263158 : 0.850614880441
0.368421052632 : 0.850614880441
0.394736842105 : 0.850614880441
0.421052631579 : 0.850614880441
0.447368421053 : 0.850614880441
0.473684210526 : 0.850614880441
0.5 : 0.850614880441


In [273]:
i = 9090
print(new_predictor[i])
print(forest_prediction[i, 1])
print(predicted_y_test[i, 1])

0.241075271487
0.241075271487
0.241075271487


Наводит на мысли. 

#Попробуем обучить отдельный классификатор по тексту

In [ ]:
cols = ['CodeLen', 'NumWords', 'NumSentences', 'Questions_ratio',  'NonWords', 'Digits', 
             'NonAlNums', 'EmptyTags', 'NumLastQuestions', 'Comma_ratio']
cols_norm = cols
cols_log = cols

x_train_text = np.array(norm_scaler(x_tr[cols_norm[0]]))[:, None]
x_test_text = np.array(norm_scaler(x_test[cols_norm[0]]))[:, None]
for col in cols_norm[1:]:
    x_train_text = np.hstack((x_train_text, np.array(norm_scaler(x_tr[col]))[:, None]))
    x_test_text = np.hstack((x_test_text, np.array(norm_scaler(x_test[col]))[:, None]))
for col in cols_log:
    x_train_text = np.hstack((x_train_text, np.array(log_scaler(x_tr[col]))[:, None]))
    x_test_text = np.hstack((x_test_text, np.array(log_scaler(x_test[col]))[:, None]))

In [282]:
x_train_text = sps.csr_matrix(sps.hstack((x_train_2, tag_tr_tfidf, title_tr_tfidf, text_tr_tfidf, code_tr_tfidf, 
                                       small_bigrams_tr_tfidf, bititle_tr)))
x_test_text = sps.csr_matrix(sps.hstack((x_test_2, tag_test_tfidf, title_test_tfidf, text_test_tfidf, code_test_tfidf, 
                                      small_bigrams_test_tfidf, bititle_test)))

In [283]:
clf_linear = LogisticRegression()
clf_linear.fit(x_train_text, y_tr)
test_y_test = clf_linear.predict_proba(x_test_text)
print(roc_auc_score(y_test, test_y_test[:, 1]))

0.849273605176


In [ ]:
for w in np.linspace(0, 1, 10):
    print(roc_auc_score(y_test, w * text_y_test[:, 1] + (1 - w) * predicted_y_test[:, 1]))

#Попробуем построить сложный алгоритм, который будет использовать результаты Random Forest и логистической регрессии

Попозже

#Попробуем выделить важные теги и слова в текстах и заголовках

##Функции для выделения словаря оптимальных слов и построения разреженной матрицы по данному словарю

In [203]:
def choose_vocabulary(pos_df, neg_df, new_vocab_len):
    word_fd = FreqDist()
    label_word_fd = ConditionalFreqDist()
    
    for text in pos_df:
        for word in text:
            word_fd[word.lower()] += 1
            label_word_fd['pos'][word.lower()] += 1
    
    for text in neg_df:
        for word in text:
            word_fd[word.lower()] += 1
            label_word_fd['neg'][word.lower()] += 1
    
    pos_word_count = label_word_fd['pos'].N()
    neg_word_count = label_word_fd['neg'].N()
    total_word_count = pos_word_count + neg_word_count
    
    word_scores = {}
    for word, freq in word_fd.items():
        pos_score = BigramAssocMeasures.chi_sq(label_word_fd['pos'][word],
            (freq, pos_word_count), total_word_count)
        neg_score = BigramAssocMeasures.chi_sq(label_word_fd['neg'][word],
            (freq, neg_word_count), total_word_count)
        word_scores[word] = pos_score + neg_score
    
    best = sorted(word_scores.items(), key=operator.itemgetter(1))
    bestwords = set([w for w, s in best[-new_vocab_len:]])
    vocabulary = {}
    for word in list(bestwords):
        vocabulary.setdefault(word, len(vocabulary))
    return vocabulary, best

In [204]:
def text_to_sparse_mat(df, vocabulary, shape=None):
    indptr, indices, dt = [0], [], []
    for text in df:
        for word in text:
            if word in vocabulary.keys():
                index = vocabulary[word]
                indices.append(index)
                dt.append(1)
        indptr.append(len(indices))
    if shape is None:
        mat = sps.csr_matrix((dt, indices, indptr), dtype=float)
    else:
        mat = sps.csr_matrix((dt, indices, indptr), dtype=float, shape=shape)
    return mat

#Теги

In [221]:
vocabulary, best_words = choose_vocabulary(x_tr.Tags[y_tr == 1], x_tr.Tags[y_tr == 0], 500)
tags_tr_mat = text_to_sparse_mat(x_tr.Tags, vocabulary)
tags_test_mat = text_to_sparse_mat(x_test.Tags, vocabulary, shape=(len(y_test), len(vocabulary)))

In [222]:
transformer = TfidfTransformer()
small_tag_tr_tfidf = transformer.fit_transform(tags_tr_mat)
small_tag_test_tfidf = transformer.fit_transform(tags_test_mat)

In [223]:
best_words[-10:]

[('interview-questions', 314.0751636471195),
 ('algorithm', 326.90989417952403),
 ('java', 353.92960766514267),
 ('career-development', 379.6728284041657),
 ('c', 390.37672085176735),
 ('ubuntu', 420.09236241492135),
 ('programming-languages', 442.9992701820673),
 ('linux', 655.0426960173544),
 ('books', 824.445753199567),
 ('php', 1089.6837563029196)]

#Стемминг и очистка текста

In [323]:
stop_words = stopwords.words('english') + ['this', 'doesn\'t', 'would', 'could', 'don\'t', 'im']

In [242]:
data['StemmedText'] = data.PureText

In [243]:
stemmer = nltk.stem.porter.PorterStemmer()
data.PureText = data.PureText.apply(lambda text: [stemmer.stem(word) for word in text])

In [244]:
train_data = data[:70000]
test_data = data[70000:]
train_columns = [col for col in data.columns if col != 'OpenStatus' and col != 'PostClosedDate']
x_tr = train_data[train_columns]
y_tr = train_data.OpenStatus
x_test = test_data[train_columns]
y_test = test_data.OpenStatus

In [324]:
vocabulary, best_words = choose_vocabulary(x_tr.StemmedText[y_tr == 1], x_tr.StemmedText[y_tr == 0], 1000)
text_tr_mat = text_to_sparse_mat(x_tr.StemmedText, vocabulary)
text_test_mat = text_to_sparse_mat(x_test.StemmedText, vocabulary, shape=(len(y_test), len(vocabulary)))

In [325]:
transformer = TfidfTransformer()
small_text_tr_tfidf = transformer.fit_transform(text_tr_mat)
small_text_test_tfidf = transformer.fit_transform(text_test_mat)

In [247]:
best_words[-10:]

[('java', 2222.123327785344),
 ('me', 2304.4035165418104),
 ('good', 2572.877843878534),
 ('book', 2830.589559029197),
 ('languag', 3275.8731603279043),
 ('you', 3421.8325956676185),
 ('program', 3454.8842428873977),
 ('learn', 4147.922819978187),
 ('develop', 4511.108451246573),
 ('the', 12125.870559098008)]

#Title

In [205]:
vocabulary, best_words = choose_vocabulary(x_tr.Title[y_tr == 1], x_tr.Title[y_tr == 0], 1000)
title_tr_mat = text_to_sparse_mat(x_tr.Title, vocabulary)
title_test_mat = text_to_sparse_mat(x_test.Title, vocabulary, shape=(len(y_test), len(vocabulary)))

In [206]:
best_words[-20:]

[('java', 439.2368387545045),
 ('website', 444.67590741344395),
 ('learning', 449.3334246997803),
 ('you', 451.3187663132005),
 ('code', 473.77039880059004),
 ('which', 480.15194838052946),
 ('development', 504.7113641507135),
 ('book', 511.2794781094807),
 ('php', 519.4437862586891),
 ('language', 529.4914582018488),
 ('learn', 535.9997967718931),
 ('software', 604.9853749943705),
 ('books', 714.4920847253122),
 ('good', 774.8362294555498),
 (':', 780.0319335366202),
 ('for', 808.1749242749489),
 ('?', 908.4750776240634),
 ('best', 1041.257102693981),
 ('programming', 1106.022668371447),
 ('what', 1726.5704046452045)]

In [207]:
transformer = TfidfTransformer()
small_title_tr_tfidf = transformer.fit_transform(title_tr_mat)
small_title_test_tfidf = transformer.fit_transform(title_test_mat)

#Code

In [208]:
vocabulary, best_words = choose_vocabulary(x_tr.SplittedCode[y_tr == 1], x_tr.SplittedCode[y_tr == 0], 500)
code_tr_mat = text_to_sparse_mat(x_tr.SplittedCode, vocabulary)
code_test_mat = text_to_sparse_mat(x_test.SplittedCode, vocabulary, shape=(len(y_test), len(vocabulary)))

In [209]:
transformer = TfidfTransformer()
small_code_tr_tfidf = transformer.fit_transform(code_tr_mat)
small_code_test_tfidf = transformer.fit_transform(code_test_mat)

In [210]:
best_words[-20:]

[('boxman', 747.8743130910759),
 ('idx', 776.4194394154746),
 ('workspaceindex', 786.5616544163311),
 ('xmm', 788.8099783522462),
 ('debug', 802.3319799826629),
 ('class', 810.5540618676066),
 ('modules', 820.7817524321819),
 ('vs', 878.7584335924881),
 ('lt', 895.9131038475962),
 ('cout', 901.3908676572921),
 ('int', 977.7950895646475),
 ('q', 1002.8959898074042),
 ('option', 1115.8142364632117),
 ('self', 1122.9739526132153),
 ('if', 1161.5437299044045),
 ('field', 1243.0812518767098),
 ('id', 1705.856240961436),
 ('n', 1838.6451171125552),
 ('j', 1839.0213296812312),
 ('i', 3426.584959334477)]

#Bigrams

In [211]:
def choose_bigram_vocabulary(pos_df, neg_df, new_vocab_len):
    word_fd = FreqDist()
    label_word_fd = ConditionalFreqDist()
    
    for text in pos_df:
        bigrams_lst = list(nltk.bigrams(text))
        for bigram in bigrams_lst:
            word_fd[bigram] += 1
            label_word_fd['pos'][bigram] += 1
    
    for text in neg_df:
        bigrams_lst = list(nltk.bigrams(text))
        for bigram in bigrams_lst:
            word_fd[bigram] += 1
            label_word_fd['neg'][bigram] += 1
    
    pos_word_count = label_word_fd['pos'].N()
    neg_word_count = label_word_fd['neg'].N()
    total_word_count = pos_word_count + neg_word_count
    
    word_scores = {}
    for word, freq in word_fd.items():
        pos_score = BigramAssocMeasures.chi_sq(label_word_fd['pos'][word],
            (freq, pos_word_count), total_word_count)
        neg_score = BigramAssocMeasures.chi_sq(label_word_fd['neg'][word],
            (freq, neg_word_count), total_word_count)
        word_scores[word] = pos_score + neg_score
    
    best = sorted(word_scores.items(), key=operator.itemgetter(1))
#     print(best[-100:])
#     for elem in best[-100:]:
#         word = elem[0]
#         print(elem[0])
#         print('Positive:', label_word_fd['pos'][word])
#         print('Negative:', label_word_fd['neg'][word])
    bestwords = set([w for w, s in best[-new_vocab_len:]])
    vocabulary = {}
    for word in list(bestwords):
        vocabulary.setdefault(word, len(vocabulary))
    return vocabulary, list(best)

In [212]:
def text_to_sparse_mat_bigrams(df, vocabulary, shape=None):
    indptr, indices, dt = [0], [], []
    for text in df:
        bigrams_lst = list(nltk.bigrams(text))
        for bigram in bigrams_lst:
            if bigram in vocabulary.keys():
                index = vocabulary[bigram]
                indices.append(index)
                dt.append(1)
        indptr.append(len(indices))
    if shape is None:
        mat = sps.csr_matrix((dt, indices, indptr), dtype=float)
    else:
        mat = sps.csr_matrix((dt, indices, indptr), dtype=float, shape=shape)
    return mat

In [213]:
vocabulary, best_words = choose_bigram_vocabulary(x_tr.PureText[y_tr == 1], x_tr.PureText[y_tr == 0], 1000)
bigrams_tr_mat = text_to_sparse_mat_bigrams(x_tr.PureText, vocabulary, shape=(len(y_tr), len(vocabulary)))
bigrams_test_mat = text_to_sparse_mat_bigrams(x_test.PureText, vocabulary, shape=(len(y_test), len(vocabulary)))

In [215]:
transformer = TfidfTransformer()
small_bigrams_tr_tfidf = transformer.fit_transform(bigrams_tr_mat)
small_bigrams_test_tfidf = transformer.fit_transform(bigrams_test_mat)

In [214]:
best_words[-1000:-950]

[(('the', 'dictionary'), 69.49041464069393),
 (('the', 'browser'), 69.52309008089568),
 (('who', 'is'), 69.55732708391982),
 (('I', 'writing'), 69.61122035342913),
 (('the', 'repository'), 69.63022579745427),
 (('to', 'query'), 69.88553813315752),
 (('should', 'learn'), 69.93470085470358),
 (('learn', 'C'), 69.93470085470358),
 (('Any', 'help'), 69.94191527099798),
 (('the', 'custom'), 69.98155996058946),
 (('ajax', 'call'), 70.04002988260294),
 (('know', 'of'), 70.04595430356127),
 (('the', 'label'), 70.11201347065148),
 (('an', 'attribute'), 70.12441847633264),
 (('the', 'screen'), 70.32934743071465),
 (('app', 'for'), 70.34565158892647),
 (('an', 'IDE'), 70.46175588090502),
 (('an', 'ajax'), 70.48645652847995),
 (('this', 'object'), 70.48645652847995),
 (('event', 'I'), 70.51906015646328),
 (('the', 'advantages'), 70.62494262276083),
 (('select', 'the'), 70.62693831284417),
 (('which', 'contains'), 70.63849195711943),
 (('like', 'me'), 70.71781013802234),
 (('the', 'validation'), 70

In [83]:
small_bigrams_tr_tfidf

<70000x200000 sparse matrix of type '<class 'numpy.float64'>'
	with 2876665 stored elements in Compressed Sparse Row format>

#Nonpure 1-grams

In [216]:
vocabulary, best_words = choose_vocabulary(x_tr.Text[y_tr == 1], x_tr.Text[y_tr == 0], 5000)
text_tr_mat = text_to_sparse_mat(x_tr.Text, vocabulary)
text_test_mat = text_to_sparse_mat(x_test.Text, vocabulary, shape=(len(y_test), len(vocabulary)))

In [217]:
transformer = TfidfTransformer()
small_text_tr_tfidf = transformer.fit_transform(text_tr_mat)
small_text_test_tfidf = transformer.fit_transform(text_test_mat)

In [218]:
best_words[-20:]

[('books', 1462.850206555511),
 (':', 1490.1487464243346),
 ('php', 1590.1988654592546),
 ('method', 1610.6595750204729),
 ('for', 1728.1543094475685),
 ('when', 1876.0520847435466),
 ('please', 1888.2981386789754),
 ('about', 1947.4377722133913),
 ('software', 2070.724757923574),
 ('this', 2097.8032076389777),
 ('language', 2130.854985814294),
 ('java', 2181.0572161908244),
 ('me', 2230.1357531661906),
 ('.chr', 2257.6931456752004),
 ('learn', 2489.0656426422465),
 ('good', 2518.4861282018223),
 ('programming', 3249.8915994023428),
 ('you', 3326.8090454118355),
 ('`', 3355.5555988189917),
 ('the', 12623.372177001089)]

#Nonpure text bigrams

In [681]:
vocabulary, best_words = choose_bigram_vocabulary(x_tr.Text[y_tr == 1], x_tr.Text[y_tr == 0], 10000)
bigrams_tr_mat = text_to_sparse_mat(x_tr.Text, vocabulary, shape=(len(y_tr), len(vocabulary)))
bigrams_test_mat = text_to_sparse_mat(x_test.Text, vocabulary, shape=(len(y_test), len(vocabulary)))

In [682]:
transformer = TfidfTransformer()
small_bigrams_tr_tfidf = transformer.fit_transform(bigrams_tr_mat)
small_bigrams_test_tfidf = transformer.fit_transform(bigrams_test_mat)

In [683]:
best_words[-20:]

[(('a', 'way'), 793.1271749248762),
 (('help', 'me'), 852.4774748878591),
 (('have', 'a'), 852.7846641452204),
 (('trying', 'to'), 855.6013928668348),
 (('to', 'the'), 912.6264073186763),
 (('looking', 'for'), 928.4672539363041),
 (('like', 'this'), 939.4017176257694),
 ((':', 'I'), 996.7627375733846),
 (('I', 'get'), 1052.383029724372),
 ((',', "''"), 1054.2253180850128),
 ((')', '`'), 1067.4304354418412),
 (('way', 'to'), 1111.4876339020989),
 (('I', 'have'), 1170.2207655952197),
 (('\\', "''"), 1187.7056529571526),
 (('this', ':'), 1195.2651152235876),
 (('in', 'the'), 1201.5857565380602),
 (('the', 'following'), 1269.634177474195),
 (('to', 'learn'), 1672.0846783804823),
 ((')', '.chr'), 2262.9662823286635),
 (('.chr', '('), 2262.9662823286635)]

#Части Речи

In [712]:
vocabulary, best_words = choose_vocabulary(x_tr.PartsOfSpeach[y_tr == 1], x_tr.PartsOfSpeach[y_tr == 0], 100)
pos_tr_mat = text_to_sparse_mat(x_tr.PartsOfSpeach, vocabulary, shape=(len(y_tr), len(vocabulary)))
pos_test_mat = text_to_sparse_mat(x_test.PartsOfSpeach, vocabulary, shape=(len(y_test), len(vocabulary)))

In [713]:
best_words[-10:]

[('md', 458.8694462115534),
 ('rbr', 469.6399491999696),
 ('jjr', 584.788319683434),
 ('cc', 586.5775317933153),
 ('jjs', 830.8957212127995),
 ('nnp', 944.1987442821198),
 ('jj', 1157.4073944038716),
 ('wp', 1164.7416171727941),
 ('vbz', 1284.2941144558347),
 ('dt', 9336.416040942027)]

In [715]:
transformer = TfidfTransformer()
pos_tr_tfidf = transformer.fit_transform(pos_tr_mat)
pos_test_tfidf = transformer.fit_transform(pos_test_mat)

#Пунктуация

In [412]:
vocabulary = choose_vocabulary(x_tr.Punctuation[y_tr == 1], x_tr.Punctuation[y_tr == 0], 500)
punct_tr_mat = text_to_sparse_mat(x_tr.Punctuation, vocabulary, shape=(len(y_tr), len(vocabulary)))
punct_test_mat = text_to_sparse_mat(x_test.Punctuation, vocabulary, shape=(len(y_test), len(vocabulary)))

AttributeError: 'DataFrame' object has no attribute 'Punctuation'

In [82]:
transformer = TfidfTransformer()
punct_tr_tfidf = transformer.fit_transform(text_tr_mat)
punct _test_tfidf = transformer.fit_transform(text_test_mat)

SyntaxError: invalid syntax (<ipython-input-82-bd0c9bd790c8>, line 3)

In [101]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)


NameError: name 'CountVectorizer' is not defined

##Отбираем слова в PureText

In [85]:
vocabulary = choose_vocabulary(x_tr.PureText[y_tr == 1], x_tr.PureText[y_tr == 0], 100000)
text_tr_mat = text_to_sparse_mat(x_tr.PureText, vocabulary, shape=(len(y_tr), len(vocabulary)))
text_test_mat = text_to_sparse_mat(x_test.PureText, vocabulary, shape=(len(y_test), len(vocabulary)))

In [86]:
transformer = TfidfTransformer()
text_tr_tfidf = transformer.fit_transform(text_tr_mat)
text_test_tfidf = transformer.fit_transform(text_test_mat)

#А что если просто по тексту?

In [90]:
vocabulary = choose_vocabulary(x_tr.Text[y_tr == 1], x_tr.Text[y_tr == 0], 100000)
text_tr_mat = text_to_sparse_mat(x_tr.Text, vocabulary, shape=(len(y_tr), len(vocabulary)))
text_test_mat = text_to_sparse_mat(x_test.Text, vocabulary, shape=(len(y_test), len(vocabulary)))

In [91]:
transformer = TfidfTransformer()
text_tr_tfidf = transformer.fit_transform(text_tr_mat)
text_test_tfidf = transformer.fit_transform(text_test_mat)

#Отбираем слова в Title

In [103]:
vocabulary = choose_vocabulary(x_tr.PureText[y_tr == 1], x_tr.PureText[y_tr == 0], 500)
text_tr_mat = text_to_sparse_mat(x_tr.PureText, vocabulary, shape=(len(y_tr), len(vocabulary)))
text_test_mat = text_to_sparse_mat(x_test.PureText, vocabulary, shape=(len(y_test), len(vocabulary)))

In [483]:
x_tr.PureText

0        In My VB.NET web page I have this standard eve...
1        I trying to install VS on D drive but when i c...
2        My IT knowledge resides mainly in SW developme...
3        I have a single-threaded Rails app running on ...
4        im trying to show printscreen image in picture...
5        How can I create a div with rounded corners an...
6        If I wanted to display a loading page while so...
7        I have an assignment to teach a team the subje...
8        Lets say I have developed a desktop applicatio...
9        Could you please develop an automated script f...
10       I have a global variable called when a user cl...
11       How to execute one or more commands and script...
12       I got a problem with ASIHTTPRequest in my iOS ...
13       I have a view that for some reason displays th...
14       I been asked to see if we can monitor the heal...
15       In normal js one can iterate the window object...
16       How to move a triangle to a new location using.

In [490]:
bigrams_tr.shape

(70000, 1090637)

In [489]:
bigrams_test.shape

(30000, 1090637)

In [499]:
np.max(bigrams_test[0].toarray())

0.19164913498331618